In [1]:
!pip install google-generativeai
!pip install gradio
!pip install PyPDF2
!pip install python-docx
!pip install pandas
!pip install numpy

In [2]:
import google.generativeai as genai
import os
import gradio as gr
from typing import Dict, List, Tuple, Optional
import json
import PyPDF2
import docx
import pandas as pd
from google.colab import userdata
from google.colab import files
import io
import re

# Configure Gemini API
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
# Use a currently available and free model
model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest')

print("✅ Libraries imported and API configured!")

✅ Libraries imported and API configured!


In [3]:
class DocumentProcessor:
    """Process uploaded policy documents (PDF, DOCX, TXT)"""

    def __init__(self):
        self.processed_documents = {}

    def extract_pdf_text(self, file_path: str) -> str:
        """Extract text from PDF file"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            return f"Error reading PDF: {str(e)}"

    def extract_docx_text(self, file_path: str) -> str:
        """Extract text from DOCX file"""
        try:
            doc = docx.Document(file_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
            return text
        except Exception as e:
            return f"Error reading DOCX: {str(e)}"

    def extract_text_file(self, file_path: str) -> str:
        """Extract text from TXT file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            return f"Error reading TXT: {str(e)}"

    def process_document(self, file_path: str, file_name: str) -> str:
        """Process document based on file extension"""
        file_extension = file_name.lower().split('.')[-1]

        if file_extension == 'pdf':
            content = self.extract_pdf_text(file_path)
        elif file_extension == 'docx':
            content = self.extract_docx_text(file_path)
        elif file_extension == 'txt':
            content = self.extract_text_file(file_path)
        else:
            return f"Unsupported file format: {file_extension}"

        # Store processed content
        self.processed_documents[file_name] = content
        return content

    def get_all_documents_text(self) -> str:
        """Get combined text from all processed documents"""
        combined_text = ""
        for doc_name, content in self.processed_documents.items():
            combined_text += f"\n=== {doc_name} ===\n{content}\n"
        return combined_text

# Initialize document processor
doc_processor = DocumentProcessor()
print("✅ Document processor initialized!")


✅ Document processor initialized!


In [4]:
ENHANCED_POLICY_DATABASE = {
    "Basic Health Plan": {
        "description": "Most affordable option for essential healthcare needs",
        "coverage": [
            "In-Patient Hospitalization (shared room)",
            "Emergency Room Services",
            "Basic Diagnostic Services (emergency-related only)",
            "Emergency Prescriptions (up to $250 annually)",
            "Ambulance Services (emergency only)"
        ],
        "eligibility": {
            "age_range": "18-55",
            "family_size": "Individual",
            "income_requirement": "No income restriction",
            "pre_existing_conditions": "Limited coverage after waiting period"
        },
        "costs": {
            "premium": 150,
            "deductible": 1500,
            "copay": {"doctor_visit": 30, "emergency": 150, "prescription": 15},
            "out_of_pocket_max": 8000
        },
        "coverage_limit": 100000,
        "network_hospitals": 2500,
        "exclusions": ["Outpatient services", "Wellness benefits", "Maternity", "Dental care", "Vision care"],
        "waiting_periods": {"pre_existing": 12, "general": 0},
        "target_audience": ["young adults", "budget-conscious", "basic coverage", "healthy individuals"],
        "key_benefits": ["Affordable premiums", "Emergency protection", "Basic hospitalization"]
    },
    "Family Health Plus Plan": {
        "description": "Comprehensive coverage for families with in-patient and out-patient services",
        "coverage": [
            "Comprehensive In-Patient Hospitalization",
            "Family Doctor Visits (10 per year per member)",
            "Specialist Consultations (15 annually)",
            "Diagnostics and Lab Tests",
            "Emergency Care and Transport",
            "Prescription Drugs (up to $500 per member)",
            "Outpatient Surgery",
            "Physical Therapy (12 sessions per year)"
        ],
        "eligibility": {
            "age_range": "All ages",
            "family_size": "2-8 members",
            "income_requirement": "No income restriction",
            "pre_existing_conditions": "Covered after waiting period"
        },
        "costs": {
            "premium": 350,
            "deductible": {"individual": 1000, "family": 3000},
            "copay": {"doctor_visit": 25, "specialist": 40, "emergency": 100, "prescription": 10},
            "out_of_pocket_max": {"individual": 6000, "family": 12000}
        },
        "coverage_limit": 500000,
        "network_hospitals": 5000,
        "exclusions": ["Fertility treatments", "Cosmetic surgery", "Alternative therapies", "Experimental treatments"],
        "waiting_periods": {"pre_existing": 6, "maternity": 9, "general": 0},
        "target_audience": ["families", "children", "comprehensive coverage", "working professionals"],
        "key_benefits": ["Family coverage", "Outpatient services", "Comprehensive benefits", "Large network"]
    },
    "Comprehensive Health & Wellness Plan": {
        "description": "Holistic health support with wellness resources and preventive care",
        "coverage": [
            "All Family Health Plus benefits",
            "Preventive Care Services (annual checkups, screenings)",
            "Mental Health Support (12 sessions per year)",
            "Wellness Programs (gym membership, nutrition counseling)",
            "Additional Specialist Coverage (unlimited)",
            "Extended Prescription Drugs (up to $1,000 per member)",
            "Alternative Medicine (acupuncture, chiropractic)",
            "Health Coaching Services"
        ],
        "eligibility": {
            "age_range": "All ages",
            "family_size": "Individual or family",
            "income_requirement": "No income restriction",
            "pre_existing_conditions": "Full coverage after waiting period"
        },
        "costs": {
            "premium": 500,
            "deductible": {"individual": 750, "family": 2000},
            "copay": {"doctor_visit": 20, "specialist": 30, "emergency": 75, "prescription": 5},
            "out_of_pocket_max": {"individual": 5000, "family": 10000}
        },
        "coverage_limit": 1000000,
        "network_hospitals": 7500,
        "exclusions": ["Experimental treatments", "Cosmetic surgery for aesthetic purposes"],
        "waiting_periods": {"pre_existing": 6, "mental_health": 3, "general": 0},
        "target_audience": ["wellness-focused", "preventive care", "mental health", "comprehensive benefits"],
        "key_benefits": ["Wellness programs", "Mental health coverage", "Preventive care", "Low copays"]
    },
    "Senior Health Security Plan": {
        "description": "Specialized coverage for seniors with extended healthcare needs",
        "coverage": [
            "Comprehensive Hospitalization with private rooms",
            "Unlimited Specialized Consultations",
            "Routine Prescriptions (up to $2,000 annually)",
            "Vision and Dental Benefits",
            "Senior Wellness Programs",
            "Home Healthcare Services",
            "Medical Equipment Coverage",
            "Chronic Disease Management"
        ],
        "eligibility": {
            "age_range": "55+",
            "family_size": "Individual or couple",
            "income_requirement": "No income restriction",
            "pre_existing_conditions": "Full coverage with minimal waiting period"
        },
        "costs": {
            "premium": 600,
            "deductible": 1000,
            "copay": {"doctor_visit": 15, "specialist": 25, "emergency": 50, "prescription": 5},
            "out_of_pocket_max": 4000
        },
        "coverage_limit": 750000,
        "network_hospitals": "5000+ with senior-focused facilities",
        "exclusions": ["Cosmetic treatments", "Experimental treatments"],
        "waiting_periods": {"pre_existing": 3, "general": 0},
        "target_audience": ["seniors", "55+", "specialized care", "chronic conditions"],
        "key_benefits": ["Senior-focused care", "Low out-of-pocket costs", "Comprehensive coverage", "Specialized network"]
    }
}

# Enhanced Add-on Options
ENHANCED_ADD_ON_OPTIONS = {
    "Dental & Vision Add-On": {
        "cost": 50,
        "coverage": [
            "Two dental exams per year",
            "One dental cleaning per year",
            "Annual eye exam",
            "$200 limit for corrective lenses",
            "Basic dental procedures (50% coverage)"
        ],
        "eligibility": "Available with all plans",
        "waiting_period": 3
    },
    "Enhanced Maternity & Newborn Care": {
        "cost": 75,
        "coverage": [
            "Pre-natal and post-natal care",
            "Delivery expenses (normal and C-section)",
            "Newborn immunizations",
            "Breastfeeding support",
            "Pediatric care for first year"
        ],
        "eligibility": "Family Health Plus and Comprehensive Health & Wellness Plans only",
        "waiting_period": 9
    },
    "International Travel Medical Insurance": {
        "cost": 40,
        "coverage": [
            "Emergency medical coverage abroad up to $100,000",
            "24/7 global helpline",
            "Emergency evacuation coverage",
            "Trip interruption due to medical emergency"
        ],
        "eligibility": "Available with all plans",
        "waiting_period": 0
    },
    "Critical Illness Add-On": {
        "cost": 60,
        "coverage": [
            "Lump sum payment for critical illnesses",
            "Coverage for cancer, heart attack, stroke",
            "Additional support for treatment costs",
            "Income replacement during treatment"
        ],
        "eligibility": "Available with Family Health Plus, Comprehensive, and Senior plans",
        "waiting_period": 6
    }
}

print("✅ Enhanced policy database created!")


✅ Enhanced policy database created!


In [5]:
class AdvancedHealthcarePolicyAgent:
    def __init__(self, doc_processor: DocumentProcessor, model):
        self.model = model
        self.doc_processor = doc_processor
        self.conversation_history = []
        self.user_profile = {}

    def extract_user_requirements(self, user_input: str) -> Dict:
        """Enhanced user requirement extraction with better analysis"""

        # Get context from processed documents if available
        document_context = self.doc_processor.get_all_documents_text()
        context_prompt = f"\nAdditional policy information from uploaded documents:\n{document_context}" if document_context else ""

        prompt = f"""
        You are an expert healthcare insurance analyst. Analyze the user's input and extract detailed requirements.

        User Input: "{user_input}"
        {context_prompt}

        Extract and categorize the following information. If information is not provided, mark as "Not specified":

        DEMOGRAPHICS:
        - Age (exact age or age range)
        - Gender (if mentioned)
        - Location (if mentioned)

        FAMILY STRUCTURE:
        - Family status (individual, couple, family)
        - Number of dependents
        - Ages of dependents (if mentioned)
        - Spouse information (if applicable)

        HEALTH REQUIREMENTS:
        - Current health status
        - Pre-existing conditions
        - Specific medical needs
        - Prescription medications
        - Ongoing treatments

        COVERAGE PREFERENCES:
        - Preferred coverage type (basic, comprehensive, premium)
        - Specific benefits needed (dental, vision, maternity, mental health)
        - Network preferences
        - Doctor/hospital preferences

        FINANCIAL CONSIDERATIONS:
        - Budget range for premiums
        - Deductible preferences
        - Copay preferences
        - Out-of-pocket maximum tolerance

        SPECIAL REQUIREMENTS:
        - Travel needs
        - Wellness programs interest
        - Preventive care priorities
        - Emergency coverage needs

        Provide this analysis in a clear, structured JSON-like format.
        """

        try:
            response = self.model.generate_content(prompt)
            # Store user profile for future reference
            self.user_profile = response.text
            return response.text
        except Exception as e:
            return f"Error analyzing requirements: {str(e)}"

    def calculate_policy_score(self, policy_name: str, user_requirements: str) -> float:
        """Calculate a matching score for each policy based on user requirements"""

        prompt = f"""
        You are a healthcare policy matching expert. Calculate how well this policy matches the user's requirements.

        Policy: {policy_name}
        Policy Details: {json.dumps(ENHANCED_POLICY_DATABASE[policy_name], indent=2)}

        User Requirements: {user_requirements}

        Rate the match on a scale of 0-100 considering:
        1. Age eligibility (20 points)
        2. Family structure compatibility (20 points)
        3. Budget alignment (20 points)
        4. Coverage needs match (20 points)
        5. Special requirements fulfillment (20 points)

        Return only a single number between 0-100 representing the match score.
        """

        try:
            response = self.model.generate_content(prompt)
            # Extract numeric score from response
            score_text = response.text.strip()
            score = float(re.findall(r'\d+(?:\.\d+)?', score_text)[0])
            return min(100, max(0, score))  # Ensure score is between 0-100
        except:
            return 50.0  # Default score if calculation fails

    def generate_personalized_recommendation(self, user_requirements: str) -> str:
        """Generate highly personalized policy recommendations"""

        # Calculate scores for all policies
        policy_scores = {}
        for policy_name in ENHANCED_POLICY_DATABASE.keys():
            policy_scores[policy_name] = self.calculate_policy_score(policy_name, user_requirements)

        # Sort policies by score
        ranked_policies = sorted(policy_scores.items(), key=lambda x: x[1], reverse=True)

        # Get document context
        document_context = self.doc_processor.get_all_documents_text()
        context_info = f"\nReference from uploaded policy documents:\n{document_context[:2000]}..." if document_context else ""

        prompt = f"""
        You are a professional healthcare insurance advisor for HealthSecure Insurance Ltd.

        CUSTOMER REQUIREMENTS ANALYSIS:
        {user_requirements}

        POLICY RANKING (by compatibility score):
        {chr(10).join([f"{i+1}. {policy} (Score: {score}/100)" for i, (policy, score) in enumerate(ranked_policies)])}

        AVAILABLE POLICIES:
        {json.dumps(ENHANCED_POLICY_DATABASE, indent=2)}

        AVAILABLE ADD-ONS:
        {json.dumps(ENHANCED_ADD_ON_OPTIONS, indent=2)}
        {context_info}

        Provide a comprehensive, personalized recommendation including:

        1. **PRIMARY RECOMMENDATION**
           - Best matching policy with detailed reasoning
           - Specific benefits that address their needs
           - Cost breakdown (premium, deductible, copays)

        2. **ALTERNATIVE OPTIONS**
           - Second-best option with comparison
           - Budget-friendly alternative if applicable

        3. **RECOMMENDED ADD-ONS**
           - Specific add-ons based on their requirements
           - Cost-benefit analysis for each add-on

        4. **FINANCIAL SUMMARY**
           - Total monthly cost breakdown
           - Annual cost projections
           - Potential savings or value proposition

        5. **NEXT STEPS**
           - Enrollment process
           - Required documentation
           - Timeline for coverage start

        6. **PERSONALIZED INSIGHTS**
           - Why this recommendation fits their specific situation
           - Long-term considerations
           - Potential policy evolution as their needs change

        Write in a conversational, consultative tone. Be specific about numbers, benefits, and processes.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating recommendation: {str(e)}"

    def handle_detailed_inquiry(self, question: str, context: str) -> str:
        """Handle detailed follow-up questions with enhanced context"""

        document_context = self.doc_processor.get_all_documents_text()

        prompt = f"""
        You are a healthcare insurance specialist providing detailed information.

        CONVERSATION CONTEXT: {context}
        USER'S STORED PROFILE: {self.user_profile}
        CURRENT QUESTION: "{question}"

        POLICY DATABASE: {json.dumps(ENHANCED_POLICY_DATABASE, indent=2)}
        ADD-ON OPTIONS: {json.dumps(ENHANCED_ADD_ON_OPTIONS, indent=2)}

        DOCUMENT REFERENCE: {document_context[:1500] if document_context else "No additional documents"}

        Provide a comprehensive, accurate answer that:
        1. Directly addresses their specific question
        2. References exact policy details, costs, and coverage
        3. Considers their personal profile and previous context
        4. Provides actionable information
        5. Offers additional relevant insights they might not have considered

        Be specific with numbers, procedures, and timelines where applicable.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error handling inquiry: {str(e)}"

# Initialize the advanced agent
print("✅ Initializing Advanced Healthcare Policy Agent...")
advanced_agent = AdvancedHealthcarePolicyAgent(doc_processor, model)
print("✅ Advanced Healthcare Policy Agent initialized!")

✅ Initializing Advanced Healthcare Policy Agent...
✅ Advanced Healthcare Policy Agent initialized!


In [6]:

def upload_and_process_documents():
    """Handle document upload and processing"""
    print("📁 Upload your policy documents (PDF, DOCX, or TXT files)...")
    uploaded = files.upload()

    processed_files = []
    for filename, file_content in uploaded.items():
        # Save uploaded file
        with open(filename, 'wb') as f:
            f.write(file_content)

        # Process the document
        print(f"📄 Processing {filename}...")
        content = doc_processor.process_document(filename, filename)
        processed_files.append(filename)
        print(f"✅ {filename} processed successfully!")

    print(f"\n🎉 Successfully processed {len(processed_files)} document(s):")
    for file in processed_files:
        print(f"   - {file}")

    return processed_files

In [7]:

def advanced_healthcare_chatbot(message, history):
    """Enhanced chatbot with document integration and advanced recommendations"""

    if not message.strip():
        return """
        👋 Welcome to HealthSecure Insurance! I'm your personal policy advisor.

        Please tell me about your healthcare insurance needs. I can help you find the perfect policy based on:

        🔹 **Your Profile**: Age, family size, location
        🔹 **Health Needs**: Current conditions, medications, ongoing care
        🔹 **Coverage Preferences**: Budget, specific benefits needed
        🔹 **Special Requirements**: Travel, wellness, preventive care

        **Example**: "I'm 34, married with 2 kids (ages 5 and 8). Looking for comprehensive family coverage including dental and vision. Budget around $400/month."

        What would you like to know about our health insurance options?
        """

    try:
        # Determine the type of interaction
        if not history:
            # First interaction - comprehensive analysis and recommendation
            print("🔍 Analyzing your requirements...")
            user_analysis = advanced_agent.extract_user_requirements(message)

            print("💡 Generating personalized recommendations...")
            recommendation = advanced_agent.generate_personalized_recommendation(user_analysis)

            return recommendation

        else:
            # Follow-up question
            last_context = history[-1][1] if history else ""

            # Check if user wants to upload documents
            if any(word in message.lower() for word in ['upload', 'document', 'file', 'pdf']):
                return """
                📁 **Document Upload Instructions:**

                To upload your policy documents:
                1. Run the document upload cell above
                2. Select your PDF, DOCX, or TXT files
                3. I'll process them and use the information to provide better recommendations

                After uploading, ask me any questions about your policies and I'll reference the uploaded documents!
                """

            print("💬 Processing your follow-up question...")
            response = advanced_agent.handle_detailed_inquiry(message, last_context)
            return response

    except Exception as e:
        return f"""
        ❌ I apologize, but I encountered an error while processing your request.

        **Error**: {str(e)}

        Please try rephrasing your question or contact our support team for assistance.

        You can also try:
        - Being more specific about your requirements
        - Asking about a particular policy by name
        - Requesting information about add-on options
        """


In [16]:
def create_advanced_interface():
    """Create an enhanced Gradio interface with modern yellow/orange styling"""

    custom_css = """
    /* Global Styling */
    body {
        background: linear-gradient(to right, #fff3e0, #ffe0b2); /* Light orange gradient background */
        font-family: 'Arial', sans-serif;
        color: #333;
    }
    .gradio-container {
        max-width: 1000px !important;
        margin: 40px auto !important;
        border-radius: 20px;
        overflow: hidden;
        box-shadow: 0 10px 40px rgba(0,0,0,0.2);
        background-color: #ffffff; /* White background for the container */
    }

    /* Header Styling */
    .header-container {
        background: linear-gradient(135deg, #ffb300 0%, #ff8f00 100%); /* Orange gradient header */
        padding: 30px;
        color: white;
        text-align: center;
        font-size: 1.2em;
        box-shadow: 0 5px 20px rgba(0,0,0,0.1);
    }
    .header-container h1 {
        margin: 0;
        font-size: 2.8em;
        font-weight: bold;
        text_shadow: 2px 2px 4px rgba(0,0,0,0.2);
    }
     .header-container p {
        margin_top: 10px;
        font_size: 1.1em;
        opacity: 0.9;
     }


    /* Chat Interface Styling */
    .chat-container {
        padding: 20px;
    }

    /* Message Styling */
    .message {
        padding: 15px !important;
        margin: 10px 0 !important;
        border-radius: 15px !important;
        line-height: 1.6 !important;
        max_width: 80% !important; /* Limit message width */
    }

    .message.user {
        background_color: #ffe0b2 !important; /* Light orange background for user */
        align_self: flex_end !important; /* Align user messages to the right */
        margin_left: auto !important;
        border_bottom_right_radius: 5px !important; /* Pointy corner for user */
    }

    .message.bot {
        background_color: #fff9c4 !important; /* Lighter yellow background for bot */
        align_self: flex_start !important; /* Align bot messages to the left */
        margin_right: auto !important;
        border_bottom_left_radius: 5px !important; /* Pointy corner for bot */
        border_left: 4px solid #ffb300 !important; /* Orange accent border */
    }
     .avatar_image {
        width: 30px !important;
        height: 30px !important;
        border_radius: 50% !important;
        object_fit: cover !important;
    }


    /* Input Styling */
    .input_container {
        margin_top: 20px;
        padding: 15px;
        background_color: #fff8e1; /* Very light yellow background */
        border_top: 1px solid #ffb300; /* Orange border */
        display: flex;
        align_items: center;
    }

    /* Button Styling */
    .gr_button {
        background: linear_gradient(135deg, #ff9800 0%, #fb8c00 100%) !important; /* Orange button gradient */
        border: none !important;
        border_radius: 25px !important;
        padding: 10px 20px !important;
        font_weight: bold !important;
        color: white !important;
        transition: all 0.3s ease !important;
        cursor: pointer !important;
    }

    .gr_button:hover {
        background: linear_gradient(135deg, #fb8c00 0%, #ff9800 100%) !important;
        transform: translateY(-2px) !important;
        box_shadow: 0 5px 15px rgba(255, 152, 0, 0.4) !important;
    }

    /* Examples Styling */
    .examples_container {
        margin_top: 20px;
        padding: 0 20px;
    }
    .example_item {
        background_color: #fff9c4; /* Lighter yellow */
        border: 1px solid #ffecb3; /* Lighter orange border */
        border_radius: 10px;
        padding: 10px;
        margin: 5px 0;
        cursor: pointer;
        transition: background_color 0.3s ease;
    }
    .example_item:hover {
        background_color: #ffe0b2; /* Light orange on hover */
    }

    /* Stats/Info Section */
    .info_section {
        background_color: #fff3e0; /* Lightest orange background */
        padding: 20px;
        border_radius: 15px;
        margin: 20px;
        border: 1px dashed #ffb300; /* Dashed orange border */
    }
    .info_section h3 {
        color: #e65100; /* Dark orange */
        margin_top: 0;
    }

    """

    # Modern description with yellow/orange tones
    description = """
    <div class="header-container">
        <h1>🍊 HealthSecure Insurance - AI Policy Advisor 🍊</h1>
        <p>Your trusted guide to finding the perfect healthcare coverage.</p>
    </div>

    <div class="info-section">
        <h3>✨ Discover Your Ideal Plan</h3>
        <p>
        I'm here to analyze your unique needs and match you with the best health insurance policy from our selection.
        Simply tell me about your situation:
        </p>
        <ul>
            <li>👤 Who needs coverage? (e.g., single, family of 4)</li>
            <li>🎂 What are the ages?</li>
            <li>🏥 Any specific health concerns or conditions?</li>
            <li>💲 What's your budget for premiums?</li>
            <li>🦷 Do you need dental, vision, or other special benefits?</li>
        </ul>
        <p>I can also process uploaded policy documents for personalized advice!</p>
    </div>
    """

    # Enhanced example prompts
    examples = [
        "☀️ I'm a healthy 30-year-old individual looking for basic coverage under $200/month.",
        "👨‍👩‍👧‍👦 Family of 5 (3 kids under 12), need comprehensive plan with maternity option.",
        "💊 I'm 55 and manage high blood pressure, need a plan with good prescription coverage.",
        "✈️ I travel frequently for work, what plan offers international coverage?",
        "🧘‍♀️ Looking for a plan that includes wellness programs and mental health support.",
        "📈 Self-employed, need a balance of cost and coverage for myself.",
    ]

    # Create the main interface
    demo = gr.ChatInterface(
        fn=advanced_healthcare_chatbot,
        title="HealthSecure Insurance - AI Policy Advisor", # Title can be simpler as it's in the description
        description=description,
        examples=examples,
        css=custom_css,
        theme=gr.themes.Soft(
            primary_hue="orange", # Use orange hue
            secondary_hue="yellow", # Use yellow hue
            neutral_hue="gray"
        ).set(
            # Specific color overrides for a more custom look
            background_fill_primary="#fff3e0",
            background_fill_secondary="#ffe0b2",
            # Removed container_border_color as it's not supported
            button_primary_background_fill="linear-gradient(135deg, #ff9800 0%, #fb8c00 100%)",
            button_primary_text_color="white",
            button_secondary_background_fill="#fff9c4",
            button_secondary_border_color="#ffb300",
            button_secondary_text_color="#e65100",
            # Removed text_color_subdued as it's not supported
            border_color_accent="#ffb300"
        ),
        # No chatbot_kwargs needed
    )

    return demo

In [13]:

def launch_application():
    """Launch the complete healthcare policy advisor"""

    print("🚀 Launching Advanced Healthcare Policy Advisor...")
    print("=" * 60)
    print("✅ Document processor ready")
    print("✅ AI agent initialized")
    print("✅ Policy database loaded")
    print("✅ Interface created")
    print("=" * 60)

    # Create and launch the interface
    demo = create_advanced_interface()

    # Launch with enhanced settings
    demo.launch(
        share=True,           # Create public link
        debug=True,          # Enable debug mode
        server_name="0.0.0.0",  # Allow external connections
        server_port=7860,    # Default port
        show_error=True,     # Show detailed errors
        quiet=False          # Show launch information
    )

    print("\n🎉 Healthcare Policy Advisor is now running!")
    print("📱 Use the public link to share with others")
    print("📁 Run upload_and_process_documents() to add policy documents")

In [ ]:

if __name__ == "__main__":
    launch_application()

# Cell 11: Optional - Document Upload Cell (Run this to upload documents)
# Uncomment and run this cell to upload and process policy documents
"""
print("📁 Document Upload Interface")
print("=" * 40)
processed_files = upload_and_process_documents()
print(f"✅ Ready to provide enhanced recommendations based on {len(processed_files)} uploaded documents!")
"""

🚀 Launching Advanced Healthcare Policy Advisor...
✅ Document processor ready
✅ AI agent initialized
✅ Policy database loaded
✅ Interface created


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ad58d53a501d81ea0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔍 Analyzing your requirements...
💡 Generating personalized recommendations...
🔍 Analyzing your requirements...
💡 Generating personalized recommendations...
